# 📊 Analyse du marché des services de création de sites web - ComeUp

## 🎯 Objectif du projet

Cette analyse vise à comprendre le marché des services de création de sites vitrine sur la plateforme **ComeUp**. L'objectif est d'identifier les tendances technologiques, les catégories de services les plus populaires, et les stratégies tarifaires des prestataires.

## 📁 Structure des données

- **Source** : Dataset ComeUp (`../../../../data/raw/site-vitrine/comeup.csv`)
- **Taille initiale** : 1647 services analysés
- **Format final** : Dataset nettoyé et enrichi sauvegardé dans `../../../../data/processed/site-vitrine/`

## 🔍 Méthodologie d'analyse

### 1. **Nettoyage des données** 🧹
- Suppression des colonnes non pertinentes
- Traitement des valeurs manquantes
- Normalisation des prix et des notes
- Standardisation des formats de données

### 2. **Catégorisation des services** 🏷️
Classification automatique en 11 catégories principales :
- 🌐 Website Development
- 🔍 SEO Optimization  
- 📚 E-learning Platform
- 🛒 E-commerce
- 📅 Booking/Reservation
- 🎨 Portfolio/Personal
- ✏️ Blog/Content
- 🚀 Landing Page
- 🔄 Website Redesign
- 🏢 Industry-Specific Website
- ❓ Other

### 3. **Analyse technologique** ⚙️
Détection automatique de **34 technologies** différentes dans les descriptions de services :
- CMS (WordPress, Drupal, Joomla)
- Builders (Elementor, Divi, Webflow)
- Frameworks (React, Vue, Laravel, Django)
- Langages (HTML, CSS, JavaScript, PHP, Python)
- Outils design (Figma, Adobe XD, PSD)

## 📈 Principales métriques analysées

- **Distribution des prix** par catégorie et technologie
- **Corrélation notes/nombre d'évaluations**
- **Technologies les plus populaires**
- **Combinaisons technologiques fréquentes**
- **Analyse croisée** catégories vs technologies

## 🎨 Visualisations à venir

Ce notebook servira de base pour créer des visualisations approfondies permettant de :
- Identifier les niches les plus rentables
- Analyser la concurrence par technologie
- Comprendre les stratégies tarifaires optimales
- Détecter les tendances émergentes du marché

---
*Dataset final prêt pour l'analyse statistique et la visualisation* ✅

In [5]:
# Import necessary libraries
import pandas as pd
from collections import Counter
import re

In [6]:
# Set data directory path and load the dataset
data_raw_dir = '../../../data/raw/'
site_vitrine_raw_dir = data_raw_dir + 'site-vitrine/'
all_df = pd.read_csv(site_vitrine_raw_dir + "comeup.csv")

In [7]:
# Copie du DataFrame principal
df = all_df.copy()

# Suppression des colonnes avec des valeurs non pertinentes ou nulles
df = df.drop(columns=['colorGrey350 2', 'colorGrey350 3', 'colorGrey200', 'colorGrey200 2', 'colorGrey200 3', 'affiche'])

# Info sur les colonnes supprimées
print(f"Colonnes restantes: {df.columns.tolist()}")
print(f"Shape après suppression: {df.shape}")

Colonnes restantes: ['d-block src', 'me-3 src', 'vendeur', 'note', 'nb-rate', 'description', 'stretched-link href', 'mb-1']
Shape après suppression: (1647, 8)


In [8]:
# Renommage des colonnes pour une meilleure lisibilité
df = df.rename(columns={
    'd-block src': 'Image_URL',
    'me-3 src': 'Vendor_Image_URL',
    'vendeur': 'Vendor_Name',
    'description': 'Service_Description',
    'stretched-link href': 'Service_Link',
    'mb-1': 'Price',
    'note': 'Rating',
    'nb-rate': 'Number_of_Ratings'
})

print("Colonnes renommées:")
print(df.columns.tolist())

Colonnes renommées:
['Image_URL', 'Vendor_Image_URL', 'Vendor_Name', 'Rating', 'Number_of_Ratings', 'Service_Description', 'Service_Link', 'Price']


In [9]:
# Vérification des valeurs manquantes avant nettoyage
print("Valeurs manquantes par colonne:")
print(df.isnull().sum())
print(f"\nNombre total de lignes: {len(df)}")

Valeurs manquantes par colonne:
Image_URL                 0
Vendor_Image_URL          0
Vendor_Name               0
Rating                 1128
Number_of_Ratings      1128
Service_Description       0
Service_Link              0
Price                     0
dtype: int64

Nombre total de lignes: 1647


In [10]:
# Suppression des lignes avec des valeurs manquantes dans les colonnes critiques
colonnes_critiques = ['Service_Description', 'Price']
df = df.dropna(subset=colonnes_critiques)

print(f"Lignes restantes après suppression: {len(df)}")
print("Valeurs manquantes après nettoyage:")
print(df.isnull().sum())

Lignes restantes après suppression: 1647
Valeurs manquantes après nettoyage:
Image_URL                 0
Vendor_Image_URL          0
Vendor_Name               0
Rating                 1128
Number_of_Ratings      1128
Service_Description       0
Service_Link              0
Price                     0
dtype: int64


In [11]:
# Conversion du prix en valeur numérique
print("Exemples de prix avant nettoyage:")
print(df['Price'].head(10))

df['Price'] = df['Price'].str.extract(r'(\d+[\\.,]?\d*)').replace(',', '.', regex=True).astype(float)

print("\nPrix après nettoyage:")
print(df['Price'].head(10))
print(f"Prix min: {df['Price'].min()}, Prix max: {df['Price'].max()}")

Exemples de prix avant nettoyage:
0    À partir de 140,00 €
1    À partir de 390,00 €
2    À partir de 395,00 €
3     À partir de 50,00 €
4    À partir de 490,00 €
5    À partir de 390,00 €
6    À partir de 700,00 €
7    À partir de 120,00 €
8    À partir de 800,00 €
9    À partir de 195,00 €
Name: Price, dtype: object

Prix après nettoyage:
0    140.0
1    390.0
2    395.0
3     50.0
4    490.0
5    390.0
6    700.0
7    120.0
8    800.0
9    195.0
Name: Price, dtype: float64
Prix min: 1.0, Prix max: 995.0


In [12]:
# Nettoyage de la colonne Rating
print("Exemples de ratings avant nettoyage:")
print(df['Rating'].head(10))

df['Rating'] = df['Rating'].fillna('0').str.replace(',', '.').astype(float)

print("\nRatings après nettoyage:")
print(df['Rating'].head(10))
print(f"Rating min: {df['Rating'].min()}, Rating max: {df['Rating'].max()}")

Exemples de ratings avant nettoyage:
0    5,0
1    5,0
2    5,0
3    5,0
4    NaN
5    5,0
6    5,0
7    5,0
8    4,9
9    5,0
Name: Rating, dtype: object

Ratings après nettoyage:
0    5.0
1    5.0
2    5.0
3    5.0
4    0.0
5    5.0
6    5.0
7    5.0
8    4.9
9    5.0
Name: Rating, dtype: float64
Rating min: 0.0, Rating max: 5.0


In [13]:
# Extraction des valeurs numériques de Number_of_Ratings
print("Exemples de number_of_ratings avant nettoyage:")
print(df['Number_of_Ratings'].head(10))

df['Number_of_Ratings'] = df['Number_of_Ratings'].str.extract(r'(\d+)').astype(float).fillna(0).astype(int)

print("\nNumber_of_Ratings après nettoyage:")
print(df['Number_of_Ratings'].head(10))
print(f"Nombre d'évaluations min: {df['Number_of_Ratings'].min()}, max: {df['Number_of_Ratings'].max()}")

Exemples de number_of_ratings avant nettoyage:
0    (171)
1    (142)
2    (385)
3     (10)
4      NaN
5     (83)
6     (33)
7     (43)
8     (46)
9     (32)
Name: Number_of_Ratings, dtype: object

Number_of_Ratings après nettoyage:
0    171
1    142
2    385
3     10
4      0
5     83
6     33
7     43
8     46
9     32
Name: Number_of_Ratings, dtype: int32
Nombre d'évaluations min: 0, max: 385


In [14]:
# Reset de l'index
df = df.reset_index(drop=True)
print(f"Dataframe final shape: {df.shape}")
print("\nPremières lignes du dataframe nettoyé:")
df.head()

Dataframe final shape: (1647, 8)

Premières lignes du dataframe nettoyé:


,Image_URL,Vendor_Image_URL,Vendor_Name,Rating,Number_of_Ratings,Service_Description,Service_Link,Price
0,https://thumbor.comeup.com/EsTn4zsx4rMvyd7figX...,https://thumbor.comeup.com/mUVu6-D_k_fjpmkpHh7...,Jerome_webdesign,5.0,171,Je vais créer votre site sous Wordpress,https://comeup.com/api/stats/click/ff3dd463ff2...,140.0
1,https://thumbor.comeup.com/nANntK1xwVY8UAdLIRO...,https://thumbor.comeup.com/QfSpUO0DaPYjoMIGGxh...,UpWeb_Agency,5.0,142,"Je vais créer votre site web, vitrine WordPres...",https://comeup.com/fr/service/143078/creer-vot...,390.0
2,https://thumbor.comeup.com/I6QTBL8dkCj0DS33mRl...,https://thumbor.comeup.com/W-_IGvlC1p2f-mPTSBy...,Caroline_WordPress,5.0,385,Je vais créer votre site web WordPress personn...,https://comeup.com/fr/service/141075/creer-vot...,395.0
3,https://thumbor.comeup.com/aUORgiOzZB1wCeSXsTU...,https://thumbor.comeup.com/k1c7EOqwSJshNHpmLq3...,Netcoaching,5.0,10,Je vais créer ou faire la refonte Premium de v...,https://comeup.com/api/stats/click/4b9fb176350...,50.0
4,https://thumbor.comeup.com/CWx75k_6aZ9VYeQkQN-...,https://thumbor.comeup.com/Q9xHggj8ydgGI2wnIRB...,Monark_Studio,0.0,0,Je vais créer votre site de conciergerie Airbn...,https://comeup.com/api/stats/click/a5defe67980...,490.0


In [15]:
# Fonction de catégorisation améliorée des services
def categorize_service(description):
    """Catégorise les services selon le type d'activité proposée"""
    if pd.isna(description):
        return "Other"
    
    description_lower = description.lower()
    
    # SEO en priorité car spécialisé
    if re.search(r'\b(seo|référencement|optimis)', description_lower):
        return "SEO Optimization"
    
    # E-commerce
    elif re.search(r'\b(e-commerce|ecommerce|boutique|woocommerce|shop|vente|marketplace)', description_lower):
        return "E-commerce"
    
    # Formation/E-learning
    elif re.search(r'\b(formation|e-learning|elearning|cours|learndash|lms|tutor|podia|systeme\.io)', description_lower):
        return "E-learning Platform"
    
    # Réservation/Booking
    elif re.search(r'\b(réservation|booking|rendez-vous|amelia|airbnb|conciergerie)', description_lower):
        return "Booking/Reservation"
    
    # Portfolio/CV
    elif re.search(r'\b(portfolio|cv|personnel)', description_lower):
        return "Portfolio/Personal"
    
    # Blog/Magazine
    elif re.search(r'\b(blog|magazine|article)', description_lower):
        return "Blog/Content"
    
    # Landing page spécifique
    elif re.search(r'\b(landing|one.page|onepage)', description_lower):
        return "Landing Page"
    
    # Refonte
    elif re.search(r'\b(refonte|refaire|modifier|améliorer)', description_lower):
        return "Website Redesign"
    
    # Sites spécialisés par secteur
    elif re.search(r'\b(restaurant|taxi|vtc|plombier|artisan|médical|immobilier|fitness|gym)', description_lower):
        return "Industry-Specific Website"
    
    # Développement web général (vitrine, etc.)
    elif re.search(r'\b(site|web|vitrine|internet)', description_lower):
        return "Website Development"
    
    else:
        return "Other"

# Application de la catégorisation
df['Category'] = df['Service_Description'].apply(categorize_service)

print("Distribution des catégories:")
print(df['Category'].value_counts())

Distribution des catégories:
Category
Website Development          950
SEO Optimization             265
Website Redesign              62
E-commerce                    59
Other                         59
Landing Page                  55
Booking/Reservation           53
Blog/Content                  40
Portfolio/Personal            37
E-learning Platform           35
Industry-Specific Website     32
Name: count, dtype: int64


In [16]:
# Sélection des colonnes à conserver pour l'étude de marché
columns_to_keep = ["Vendor_Name", "Service_Description", "Price", "Rating", "Number_of_Ratings", "Category"]
df = df[columns_to_keep]

print(f"Colonnes finales: {df.columns.tolist()}")
print(f"Shape final avant analyse technologique: {df.shape}")

Colonnes finales: ['Vendor_Name', 'Service_Description', 'Price', 'Rating', 'Number_of_Ratings', 'Category']
Shape final avant analyse technologique: (1647, 6)


In [17]:
# Dictionnaire des technologies avec leurs variantes amélioré
tech_patterns = {
    'WordPress': r'\b(wordpress|wp)\b',
    'Webflow': r'\b(webflow)\b',
    'Divi': r'\b(divi)\b', 
    'Elementor': r'\b(elementor)\b',
    'Wix': r'\b(wix)\b',
    'Drupal': r'\b(drupal)\b',
    'NextJs': r'\b(nextjs|next\.js|next js)\b',
    'Framer': r'\b(framer)\b',
    'Django': r'\b(django)\b',
    'Laravel': r'\b(laravel)\b',
    'Symfony': r'\b(symfony)\b',
    'React': r'\b(react|reactjs)\b',
    'Vue': r'\b(vue|vuejs)\b',
    'Woocommerce': r'\b(woocommerce|woo commerce)\b',
    'HTML': r'\b(html|html5)\b',
    'CSS': r'\b(css|css3)\b',
    'JavaScript': r'\b(javascript|js)\b',
    'PHP': r'\b(php)\b',
    'Python': r'\b(python)\b',
    'Squarespace': r'\b(squarespace)\b',
    'Shopify': r'\b(shopify)\b',
    'Joomla': r'\b(joomla)\b',
    'Google Sites': r'\b(google sites)\b',
    'Systeme.io': r'\b(systeme\.io|système\.io)\b',
    'Podia': r'\b(podia)\b',
    'Webador': r'\b(webador)\b',
    'Bootstrap': r'\b(bootstrap)\b',
    'Tailwind': r'\b(tailwind)\b',
    'MySQL': r'\b(mysql)\b',
    'Node.js': r'\b(node\.js|nodejs)\b',
    'Figma': r'\b(figma)\b',
    'PSD': r'\b(psd)\b',
    'Adobe XD': r'\b(adobe xd|xd)\b'
}

print(f"Technologies à détecter: {list(tech_patterns.keys())}")

Technologies à détecter: ['WordPress', 'Webflow', 'Divi', 'Elementor', 'Wix', 'Drupal', 'NextJs', 'Framer', 'Django', 'Laravel', 'Symfony', 'React', 'Vue', 'Woocommerce', 'HTML', 'CSS', 'JavaScript', 'PHP', 'Python', 'Squarespace', 'Shopify', 'Joomla', 'Google Sites', 'Systeme.io', 'Podia', 'Webador', 'Bootstrap', 'Tailwind', 'MySQL', 'Node.js', 'Figma', 'PSD', 'Adobe XD']


In [18]:
def categorize_techno_improved(description):
    """Retourne toutes les technologies trouvées dans la description"""
    if pd.isna(description):
        return ['Unprecised']
    
    # Gestion du cas spécial "pas de WordPress"
    if re.search(r'\b(pas de wordpress|sans wordpress|no wordpress)\b', description, re.IGNORECASE):
        return ['No WordPress']
    
    found_techs = []
    description_lower = description.lower()
    
    for tech, pattern in tech_patterns.items():
        if re.search(pattern, description_lower, re.IGNORECASE):
            found_techs.append(tech)
    
    return found_techs if found_techs else ['Unprecised']

print("Fonction de catégorisation technologique définie")

Fonction de catégorisation technologique définie


In [19]:
# Application de la catégorisation technologique
df['Techno_List'] = df['Service_Description'].apply(categorize_techno_improved)

print("Exemples de technologies détectées:")
for i in range(min(10, len(df))):
    print(f"Service {i+1}: {df.iloc[i]['Techno_List']}")

Exemples de technologies détectées:
Service 1: ['WordPress']
Service 2: ['WordPress', 'Divi']
Service 3: ['WordPress', 'Divi', 'Elementor']
Service 4: ['Divi', 'Elementor']
Service 5: ['Unprecised']
Service 6: ['WordPress']
Service 7: ['Unprecised']
Service 8: ['WordPress', 'Wix', 'Drupal']
Service 9: ['WordPress']
Service 10: ['HTML', 'CSS', 'JavaScript', 'PHP']


In [20]:
# Création des colonnes d'analyse technologique
df['Techno_Main'] = df['Techno_List'].apply(
    lambda x: x[0] if len(x) == 1 else 'Multiple' if len(x) > 1 else 'Unprecised'
)

df['Techno_Count'] = df['Techno_List'].apply(len)
df['Is_Multi_Tech'] = df['Techno_Count'] > 1

print(f"Services avec une seule technologie: {sum(df['Techno_Count'] == 1)}")
print(f"Services multi-technologies: {sum(df['Is_Multi_Tech'])}")
print(f"Services sans technologie précisée: {sum(df['Techno_List'].apply(lambda x: x == 'Unprecised'))}")

Services avec une seule technologie: 1309
Services multi-technologies: 338
Services sans technologie précisée: 0


In [21]:
# Statistiques des technologies
all_technologies = [tech for tech_list in df['Techno_List'] for tech in tech_list]
tech_counts = Counter(all_technologies)

print("Technologies les plus utilisées:")
for tech, count in tech_counts.most_common(15):
    print(f"{tech}: {count}")

Technologies les plus utilisées:
WordPress: 836
Unprecised: 491
Elementor: 150
Divi: 119
HTML: 89
CSS: 82
JavaScript: 63
Wix: 56
Webflow: 36
Figma: 36
PHP: 35
Adobe XD: 23
PSD: 22
React: 22
Laravel: 22


In [22]:
# Analyse croisée : Service Category vs Technologies principales
print("Analyse croisée - Catégories de services vs Technologies principales:")
cross_analysis = pd.crosstab(df['Category'], df['Techno_Main'], margins=True)
cross_analysis

Analyse croisée - Catégories de services vs Technologies principales:


Techno_Main,Bootstrap,Divi,Django,Drupal,Elementor,Figma,Framer,Google Sites,HTML,Joomla,...,Squarespace,Symfony,Systeme.io,Unprecised,Webador,Webflow,Wix,Woocommerce,WordPress,All
Category,,,,,,,,,,,,,,,,,,,,,
Blog/Content,0,0,0,0,0,0,0,0,0,0,...,0,0,0,8,0,0,0,0,27,40
Booking/Reservation,0,0,0,0,0,0,0,0,0,0,...,0,0,0,45,0,0,0,0,7,53
E-commerce,0,0,0,0,0,0,0,0,0,0,...,0,0,0,21,1,0,0,1,20,59
E-learning Platform,0,0,0,0,0,0,0,0,0,0,...,0,0,6,11,0,0,0,0,13,35
Industry-Specific Website,0,0,0,0,0,0,0,0,0,0,...,0,0,0,25,0,0,0,0,6,32
Landing Page,0,0,0,1,0,0,0,0,0,0,...,0,0,0,14,0,0,1,0,30,55
Other,0,1,0,1,0,0,0,0,2,0,...,0,0,0,16,0,0,0,0,16,59
Portfolio/Personal,0,0,0,0,1,0,0,0,0,0,...,0,0,1,25,0,2,0,0,6,37
SEO Optimization,0,2,0,1,1,0,1,0,0,0,...,0,0,2,41,0,3,15,1,166,265


In [23]:
# Services multi-technologies
multi_tech_services = df[df['Is_Multi_Tech']]
print(f"Nombre de services multi-tech: {len(multi_tech_services)}")

# Combinaisons fréquentes
if len(multi_tech_services) > 0:
    combinations = df[df['Is_Multi_Tech']]['Techno_List'].apply(
        lambda x: ', '.join(sorted(x))
    ).value_counts()
    
    print("\nCombinaisons les plus fréquentes:")
    print(combinations.head())
else:
    print("Aucune combinaison de technologies trouvée")

Nombre de services multi-tech: 338

Combinaisons les plus fréquentes:
Techno_List
Elementor, WordPress          62
Divi, WordPress               41
Divi, Elementor, WordPress    37
Divi, Elementor               31
CSS, HTML, JavaScript         18
Name: count, dtype: int64


In [24]:
# Aperçu final du dataframe
print("Colonnes finales:")
print(df.columns.tolist())
print(f"\nShape final: {df.shape}")
print("\nPremières lignes:")

df.head()

Colonnes finales:
['Vendor_Name', 'Service_Description', 'Price', 'Rating', 'Number_of_Ratings', 'Category', 'Techno_List', 'Techno_Main', 'Techno_Count', 'Is_Multi_Tech']

Shape final: (1647, 10)

Premières lignes:


,Vendor_Name,Service_Description,Price,Rating,Number_of_Ratings,Category,Techno_List,Techno_Main,Techno_Count,Is_Multi_Tech
0,Jerome_webdesign,Je vais créer votre site sous Wordpress,140.0,5.0,171,Website Development,[WordPress],WordPress,1,False
1,UpWeb_Agency,"Je vais créer votre site web, vitrine WordPres...",390.0,5.0,142,SEO Optimization,"[WordPress, Divi]",Multiple,2,True
2,Caroline_WordPress,Je vais créer votre site web WordPress personn...,395.0,5.0,385,SEO Optimization,"[WordPress, Divi, Elementor]",Multiple,3,True
3,Netcoaching,Je vais créer ou faire la refonte Premium de v...,50.0,5.0,10,Website Redesign,"[Divi, Elementor]",Multiple,2,True
4,Monark_Studio,Je vais créer votre site de conciergerie Airbn...,490.0,0.0,0,Booking/Reservation,[Unprecised],Unprecised,1,False


In [26]:
data_processed_dir = "../../../data/raw/"
site_vitrine_processed_dir = data_processed_dir + 'site-vitrine/'

# Enregistrement du DataFrame nettoyé et enrichi
df.to_csv(site_vitrine_processed_dir + 'comeup-cleaned.csv', index=False)
print(f"DataFrame enregistré dans {site_vitrine_processed_dir + 'comeup-cleaned.csv'}")

DataFrame enregistré dans ../../../data/raw/site-vitrine/comeup-cleaned.csv
